# Orquestación con Prefect

Este notebook documenta y permite ejecutar el **pipeline ETL orquestado** con **Prefect**.  
La lógica del flujo vive en `scripts/etl_fixtures.py` (para producción / CLI); aquí lo consumimos para:
- Mostrar el **setup** (credenciales, rutas, estructura).
- Ejecutar el **flow** de ejemplo.
- Indicar cómo **servir** el flujo y monitorearlo desde **Prefect Cloud**.

> **Nota:** El notebook manual (`ETL_API_Football.ipynb`) NO depende de Prefect.

## 1. Configuración de entorno y rutas

In [1]:
# Este bloque asegura que exista la estructura mínima del datalake
# al correr el flujo desde el notebook (modo demo/documentación).
# Nota: en el script etl_fixtures.py las rutas se definen de nuevo
# para la ejecución real por CLI / Prefect.

import os

DATALAKE_ROOT = "data/etl_datalake"
BRONZE_FIXTURES = f"{DATALAKE_ROOT}/bronze/api_football/fixtures"
SILVER_FIXTURES = f"{DATALAKE_ROOT}/silver/api_football/fixtures"
GOLD_FIXTURES   = f"{DATALAKE_ROOT}/gold/api_football/fixtures"
EXPORTS_DIR     = f"{DATALAKE_ROOT}/exports"

for p in [BRONZE_FIXTURES, SILVER_FIXTURES, GOLD_FIXTURES, EXPORTS_DIR, "scripts"]:
    os.makedirs(p, exist_ok=True)

print("Estructura base creada/validada ✅")

Estructura base creada/validada ✅


## 2. Credenciales y configuración

In [2]:
# Se lee la configuración desde pipeline.conf
# Contiene dos secciones:
# [api-credentials] -> API-Football
# [prefect]         -> Prefect Cloud

from configparser import ConfigParser

parser = ConfigParser()
parser.read("pipeline.conf")

# --- API-Football ---
BASE_URL = parser["api-credentials"]["base_url"]
API_KEY  = parser["api-credentials"]["api_key"]

print("Config OK ✅")
print("BASE_URL:", BASE_URL)
print("API_KEY:  (oculto) ****")

Config OK ✅
BASE_URL: https://v3.football.api-sports.io
API_KEY:  (oculto) ****


## 3. Prefect: conexión y conceptos

Prefect permite **orquestar** un pipeline en pasos claros:

- **Flow** → el contenedor principal del proceso ETL.  
- **Task** → cada paso atómico dentro del flow (ej. extracción, transformación, carga).  
- **Serve / Deploy** → maneras de ejecutar y programar flows:  
  - *Serve*: correrlo como servicio local (útil en desarrollo).  
  - *Deploy*: registrarlo en Prefect Cloud para programarlo y monitorearlo.  

El login en Prefect Cloud se realiza *una sola vez desde la terminal*.
Aquí solo se muestra de manera ilustrativa.

In [3]:
# Prefect requiere login en Cloud, pero este paso se hace normalmente
# desde la terminal (una única vez por entorno).
# Aquí solo mostramos cómo se lee la API Key desde pipeline.conf.

prefect_credentials = parser["prefect"]
prefect_api_key = prefect_credentials["api_key"]

print("Prefect API Key cargada (oculta) ****")

# Login en Prefect Cloud (solo a modo ilustrativo; en práctica se hace una vez desde la terminal)
# !prefect cloud login -k {prefect_api_key}

Prefect API Key cargada (oculta) ****


In [4]:
import prefect
print("Prefect versión:", prefect.__version__)

Prefect versión: 2.20.9


## 4. Uso del flujo desde `scripts/etl_fixtures.py`

El flujo ETL está definido en `scripts/etl_fixtures.py`.  
En este notebook es posible:

- **4.1 Opción A — Corrida única (demo one-off):** ejecuta el flow una sola vez para validar la orquestación.  
- **4.2 Opción B — Servir el flow (opcional):** mantiene el flow activo como servicio local.

### 4.1 Opción A — Ejecutar una corrida orquestada (demo one-off)

In [4]:
import importlib

# Importa el script de orquestación
etl = importlib.import_module("scripts.etl_fixtures")

# Ejecuta el flujo ETL de forma local (modo recomendado dentro del Notebook)
etl.etl_parametrizable(endpoints=["fixtures"])

# Nota:
# El mismo flujo también puede ejecutarse desde la terminal:
#     python scripts/etl_fixtures.py
# o, dentro del Notebook:
#     !python scripts/etl_fixtures.py
# Todas estas opciones llaman al mismo flow definido en scripts/etl_fixtures.py.

07:06:02.682 | INFO    | prefect.engine - Created flow run 'satisfied-armadillo' for flow 'etl-parametrizable'

07:06:02.689 | INFO    | Flow run 'satisfied-armadillo' - View at https://app.prefect.cloud/account/1513bf29-3686-40b8-9dbf-c85ba6a6f8c0/workspace/377710aa-6343-48a1-a52d-8fd9be6fbba7/flow-runs/flow-run/068d9088-a7d6-7c08-8000-aabc65c0662a

07:06:03.439 | INFO    | Flow run 'satisfied-armadillo' - Created task run 'extract_data-0' for task 'extract_data'

07:06:03.454 | INFO    | Flow run 'satisfied-armadillo' - Executing 'extract_data-0' immediately...

07:06:06.755 | INFO    | Task run 'get_data-fixtures' - Finished in state Completed()

...

07:06:18.042 | INFO    | Flow run 'satisfied-armadillo' - Created task run 'transform_to_gold_from_silver-0' for task 'transform_to_gold_from_silver'

07:06:18.042 | INFO    | Flow run 'satisfied-armadillo' - Executing 'transform_to_gold_from_silver-0' immediately...

07:06:19.430 | INFO    | Task run 'to_gold-fixtures' - Finished in state Completed()

07:06:19.875 | INFO    | Flow run 'satisfied-armadillo' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `list`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`'))]

### 4.2 Opción B — Servir el flow como agente local (opcional)  
Esta modalidad deja el flujo en ejecución continua y permite gestionarlo desde Prefect Cloud (ejecuciones, logs, programación).  
⚠️ La celda quedará ocupada hasta que la detengas manualmente desde tu entorno (Interrupt/Stop del notebook).
Por defecto está comentada para evitar que se ejecute por accidente.

In [ ]:
import importlib
etl = importlib.import_module("scripts.etl_fixtures")
# etl.etl_parametrizable.serve(name="ETL-Fixtures", endpoints=["fixtures"])

## 5. Monitoreo en Prefect

El monitoreo del flujo no se realiza desde el notebook, sino desde la **UI de Prefect Cloud**.  
Una vez que el flow se ejecuta (corrida única o servido como agente), es posible:

1. **Flows** → ver el listado de flows registrados (ejemplo: `ETL-Fixtures`).  
2. **Run history** → revisar el historial de ejecuciones con sus logs, tiempos y reintentos.  
3. **Blocks** → administrar la configuración de almacenamiento o infraestructura remota si se utiliza.  
4. **Schedules** → programar ejecuciones automáticas (requiere plan pago).